In [1]:
import requests
import matplotlib.pyplot as plt
from datetime import datetime
from dagster import op, job

In [ ]:
@op
def fetch_information(coordinate_file: str, email_file: str) -> dict:
    with open(coordinate_file, 'r') as file:
        latlon = file.read().strip()
    latlon_dict = {key_value.split(': ')[0]: float(key_value.split(': ')[1])
        for key_value in latlon.split(', ')
        }
    latitude, longitude = latlon_dict['lat'], latlon_dict['lon']
    with open(email_file, 'r') as file:
        email = file.read().strip()
    return {
        'lat': latitude,
        'lon': longitude,
        'email': email
    }

@op
def fetch_metadata(lat: float, lon: float, email: str) -> dict:
    url = f'https://api.weather.gov/points/{lat},{lon}'
    headers = {"User-Agent": f"MyWeatherApp/1.0 ({email})"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

@op
def fetch_forecast(metadata: dict, email: str) -> dict:
    forecast_url = metadata['properties']['forecast']
    headers = {"User-Agent": f"MyWeatherApp/1.0 ({email})"}
    response = requests.get(forecast_url, headers=headers)
    response.raise_for_status()
    return response.json()

@op
def parse_forecast(forecast_data: dict) -> dict:
    periods = forecast_data['properties']['periods']
    temperatures = [p['temperature'] for p in periods[::2]]  # Every other entry
    times = [
        datetime.fromisoformat(p['startTime'].replace("Z", "")).strftime('%Y-%m-%d %H:%M:%S') 
        for p in periods[::2]
    ]
    return {"temperatures": temperatures, "times": times}

@op
def plot_temperature(forecast: dict):
    temperatures = forecast['temperatures']
    times = forecast['times']
    
    fig, ax = plt.subplots(figsize=(15, 6))
    ax.plot(times, temperatures)
    
    formatted_times = [time.replace(' ', '\n') for time in times]
    ax.set_xticks(times)  # Set the original x-tick positions
    ax.set_xticklabels(formatted_times, rotation=20)  # Use the formatted labels

    plt.suptitle('Seven-Day Temperature Forecast\nHayden Planetarium, American Museum of Natural History, NYC', 
                y=1.0,
                fontsize=16)
    plt.title(f'Forecast starting at {times[0]}', 
            fontstyle='italic', 
            y=1,
            fontsize=10)
    plt.grid(visible=True)

    plt.show()

@job
def weather_forecast_pipeline():
    information = fetch_information('latlon.txt', 'email.txt')
    metadata = fetch_metadata(
        lat=information['lat'], 
        lon=information['lon'], 
        email=information['email']
    )
    forecast_data = fetch_forecast(metadata=metadata, email=information['email'])
    forecast = parse_forecast(forecast_data=forecast_data)
    plot_temperature(forecast=forecast)